In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
import gc
import logging #библиотека для удобного ведения логов в Python
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import AdaBoostRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint #Ранняя остановка, КПП
from keras import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization


#settings
warnings.filterwarnings('ignore')
np.random.seed(2018) #делает непредсказуемые случайные числа

In [ ]:
test_df = pd.read_csv('../input/test_ds1.csv')
train_df = pd.read_csv('../input/train_ds1.csv')

Dealing with outliers Работа с выбросами

In [ ]:
train_df['CLV']=train_df.hist_purchase_date_diff*train_df.hist_purchase_amount_mean*train_df.hist_card_id_count
train_df['CLV1']=train_df.hist_purchase_date_diff*train_df.purchase_amount_total*train_df.hist_card_id_count
test_df['CLV']=test_df.hist_purchase_date_diff*test_df.hist_purchase_amount_mean*test_df.hist_card_id_count
test_df['CLV1']=test_df.hist_purchase_date_diff*test_df.purchase_amount_total*test_df.hist_card_id_count


# Scaling
features = [c for c in train_df.columns if c not in ['card_id', 'first_active_month', 'target', 'outliers']]
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

In [ ]:
train_columns = [c for c in train_df.columns if c not in ['card_id', 'first_active_month','target','outliers']]
#train_columns = [c for c in train_df.columns if c not in ['target','outliers']]
target = train_df['target']
del train_df['target']

In [ ]:
train_df['outliers'].sum()

In [ ]:
#logger
def get_logger():
    FORMAT = '[%(levelname)s]%(asctime)s:%(name)s:%(message)s'
    logging.basicConfig(format=FORMAT) #в настройки логера передать уровень выводимых ошибок
    logger = logging.getLogger('main')
    logger.setLevel(logging.DEBUG) # Сообщение отладочное
    return logger

In [ ]:
logger = get_logger()
DEBUG = False

params = {} 
params ['learning_rate'] = 0,003 
params ['boosting_type'] = 'gbdt' 
params [ 'target' ] = 'binary' 
params ['metric'] = 'binary_logloss' 
params ['sub_feature'] = 0.5 
params ['num_leaves'] = 10 
params ['min_data'] = 50 
params ['max_depth'] = 10
clf = lgb.train (params, d_train, 100)

In [ ]:
if DEBUG == True:
    ITERATIONS = 1
else:
    ITERATIONS = 4000
lgb1 = lgb.LGBMRegressor(num_leaves=111,
                        max_depth=9,
                        learning_rate=0.005,
                        n_estimators=ITERATIONS,
                        min_child_samples=149,
                        subsample=0.71,
                        subsample_freq=1,
                        feature_fraction=0.75,
                        reg_lambda=0.26,
                        random_state=1,
                        n_jobs=4,
                        metrics='rmse')

lgb2 = lgb.LGBMRegressor(num_leaves=200,
                        max_depth=9,
                        learning_rate=0.01,
                        n_estimators=ITERATIONS,
                        min_child_samples=40,
                        subsample=0.9,
                        subsample_freq=2,
                        feature_fraction=0.8,
                        reg_lambda=0.13,
                        random_state=2,
                        n_jobs=4,
                        metrics='rmse')

xgb1 = xgb.XGBRegressor(max_depth=9,
                       learning_rate=0.005,
                       n_estimators=ITERATIONS,
                       colsample_bytree=0.75,
                       sub_sample=0.75,
                       reg_lambda=0.15,
                       n_jobs=4,
                       random_state=3)

cb1 = cb.CatBoostRegressor(iterations=ITERATIONS, learning_rate=0.007, 
                           loss_function='RMSE', bootstrap_type='Bernoulli', 
                           depth=9, rsm=0.75, subsample=0.75, random_seed=4, reg_lambda=3)

ada1 = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=8), 
                         n_estimators=ITERATIONS, learning_rate=0.007, 
                         loss='square', random_state=2019)

In [ ]:
if DEBUG==True:
    N_FOLDS=2
else:
    N_FOLDS=5
layer1_models = [lgb1, lgb2, xgb1, cb1, ada1]
layer1_names = ['lightgbm1', 'lightgbm2', 'xgboost1', 'catboost1', 'adaboost1']

In [ ]:
#model
##model params
logger.info('Prepare model')
param = {'num_leaves': 20, #41,
         'min_data_in_leaf': 100,#30, 
         'objective':'regression',
         'max_depth': 20,#-1,
         'learning_rate': 0.008,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "categorical_feature": 1,
         "random_state": 2019}

In [ ]:
#prepare fit model with cross-validation
folds = StratifiedKFold(n_splits=7, shuffle=True, random_state=2019)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

In [ ]:
folds = KFold(n_splits=7, shuffle=True, random_state=2019)

In [ ]:
#run model
logger.info('Start running model')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df,train_df['outliers'].values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train_df.iloc[val_idx][train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print ('shape-', feature_importance_df.shape)
    #predictionstarget
    predictions += clf.predict(test_df[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    logger.info(strLog)
    
np.sqrt(mean_squared_error(oof, target))

In [ ]:
#3.6601119214463607
#3.6609737986076882
#3.658594491669437
#3.6590197344766464

In [ ]:
##plot the feature importance
logger.info("Feature importance plot")
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,26))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (averaged over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

Neural network

In [ ]:
features = [c for c in train_df.columns if c not in ['card_id', 'first_active_month', 'target', 'outliers']]

In [ ]:
target.head()

In [ ]:
# Preparation
oof_train_nn = np.zeros(shape=(len(train_df),1))
oof_test_nn = np.zeros(shape=(len(test_df),1))
scaler = StandardScaler()
scaler.fit(train_df[features])
X_train = scaler.transform(train_df.iloc[:][features].values)
X_test = scaler.transform(test_df.iloc[:][features].values)

X_train = pd.DataFrame(X_train, index=train_df[features].index, columns=train_df[features].columns)
X_test = pd.DataFrame(X_test, index=test_df[features].index, columns=test_df[features].columns)


In [ ]:
if DEBUG == True:
    EPOCHS=1
else:
    EPOCHS=30 

In [ ]:
def nn_model(input_shape):
    model = Sequential()
    model.add(Dense(128, input_dim = input_shape, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

early_stop = EarlyStopping(patience=5, verbose=True)


In [ ]:
train_df.shape, target.shape

In [ ]:
i = 0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print (i)
    i += 1

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df,train_df['outliers'].values)):
      
    print('Fold no %i/%i'%(fold_+1,N_FOLDS))
    trn_data = X_train.iloc[trn_idx][features]
    trn_label = target.iloc[trn_idx]
    val_data = X_train.iloc[val_idx][features]
    val_label = target.iloc[val_idx]
    model = nn_model(trn_data.shape[1])
    hist = model.fit(trn_data,trn_label,
                     validation_data = (val_data, val_label),
                     epochs=EPOCHS, 
                     batch_size=512, 
                     verbose=True, 
                     callbacks=[early_stop])
    oof_train_nn[val_idx,0] = model.predict(val_data)[:,0]
    oof_test_nn[:,0] += model.predict(X_test[features])[:,0]/N_FOLDS

In [ ]:
score_nn = mean_squared_error(oof_train_nn, target)**0.5
print('Training CV score for neural network: %.5f' %score)
layer1_names.append('neural_net')
layer1_score.append(score_nn)

oof_train = np.hstack((oof_train, oof_train_nn))
oof_test = np.hstack((oof_test, oof_test_nn))

In [ ]:
# Print first layer result
layer1 = pd.DataFrame()
layer1['models'] = layer1_names
layer1['CV_score'] = layer1_score
layer1


In [ ]:
##submission
logger.info("Prepare submission")
sub_df = pd.DataFrame({"card_id":test_df["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission.csv", index=False)

In [ ]:
!ls

In [ ]:
sub_df.shape#head()